In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
%cd /content/drive/MyDrive/security_mft

/content/drive/MyDrive/GitHub/security_mft


In [43]:
%pwd
%ls

code/  dfxml_files/  MFT_files/  README.md  resident_files/


In [44]:
import pandas as pd
import numpy as np
import os
import glob
import sys
import hashlib
import re
from datetime import datetime
import time

import xml.etree.ElementTree as ET


In [45]:
start_time = datetime.now()
mode = "run" #run or test
env = 1 # 1- google colab,  else - local
if(env == 1):
  pathToDxml = "/content/drive/MyDrive/security_mft/dfxml_files"
else:
  pathToDxml = "."



In [46]:
def getTag(tree):
  tree = ET.parse('first_second.dfxml')
  tagList = []
  for elem in tree.iter():
    tagList.append(re.sub(r"\{[^()]*\}", "", elem.tag))
  #tagList = list(set(tagList))
  return tagList

In [47]:
#test 1
s = '{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}gid'
s = re.sub(r'{.+}', '', s)
print(f"after: {s}")

after: gid


In [48]:
def parseDxml(pathToFile):
  start_time = datetime.now()

  tree = ET.parse(pathToFile)
  root = tree.getroot()
  df = showInfo(root)
  csv_filename = f"{pathToDxml}/parse_result_{os.path.basename(pathToFile)[:-6]}.csv"
  df.to_csv (csv_filename, index = False, header=True)

  end_time = datetime.now()
  print(f"Duration of parsing data for {os.path.basename(pathToFile)[:-6]}: {end_time - start_time}")



In [49]:
import re

# #test 1
# s = '{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}gid'
# s = re.sub(r'{.+}', '', s)
# print(f"after: {s}")

# #test 2
# s = '<@ """@$ FSDF >something something <more noise>'
# s = re.sub(r'<.+>', '',s)
# # s = re.sub('<[^>]+>', '', s)
# print(f"after: {s}")

def rename(s):
  s = re.sub(r'{.+}', '', s)
  # print(s)
  return s

In [50]:
def updt(total, progress):
    """
    Displays or updates a console progress bar.

    Original source: https://stackoverflow.com/a/15860757/1391441
    """
    barLength, status = 20, ""
    progress = float(progress) / float(total)
    if progress >= 1.:
        progress, status = 1, "\r\n"
    block = int(round(barLength * progress))
    text = "\r[{}] {:.0f}% {}".format(
        "#" * block + "-" * (barLength - block), round(progress * 100, 0),
        status)
    sys.stdout.write(text)
    sys.stdout.flush()

'''
runs = 300
for run_num in range(runs):
    time.sleep(.1)
    updt(runs, run_num + 1)
'''

'\nruns = 300\nfor run_num in range(runs):\n    time.sleep(.1)\n    updt(runs, run_num + 1)\n'

In [51]:





def showInfo(root):
  
  test = 0
  listColumns = ['fileobject_id','tag','text','attrib']
  df = pd.DataFrame(columns=listColumns)
  fileobjects = root.findall('.//{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}fileobject')
  for fileobject_id, fileobject in enumerate(fileobjects):

    for node in fileobject.iter():
      if(mode == 'tset'):
        pass
        # if(test < 1):
          # print(f"id: {fileobject_id}, tag: {rename(node.tag)}")
          # print(f"id: {fileobject_id}, text: {node.text}")
          # print(f"id: {fileobject_id}, attributes: {node.attrib}")
      tempDict = {}
      tempDict['fileobject_id'] = fileobject_id
      tempDict['tag'] = rename(node.tag)
      tempDict['text'] = node.text
      tempDict['attrib'] = node.attrib
      if(node.attrib): #if the attrib exists
        for key, item in node.attrib.items():
          # print(f"attrib: key: {key}, item: {item}")
          tempDict[rename(key)] = item
      df = df.append(tempDict, 
              ignore_index=True)


    test+=1
    time.sleep(.1)
    updt(len(fileobjects), fileobject_id + 1)
    if(mode == 'test'):
      if(test==1):
        break

  return df
      
      



In [ ]:
test = 0
for eachFile in glob.glob(f"{pathToDxml}/*.dfxml"):
  print(f"parsing {eachFile} ...")
  if (os.path.exists(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv")):
    print(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv exists, move to next dfxml file.")
    continue
  # print(f"size (bytes): {os.path.getsize(eachFile)}")
  # print(f"basename (no extention): {os.path.basename(eachFile)[:-6]}")
  parseDxml(eachFile)
  test+=1
  if(mode == "test"):
    if(test==1):
      break
  

parsing /content/drive/MyDrive/security_mft/dfxml_files/fourth_fifth.dfxml ...
/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv exists, move to next dfxml file.
parsing /content/drive/MyDrive/security_mft/dfxml_files/sixth_seventh.dfxml ...
[###-----------------] 17% 

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
filepath = glob.glob(f"{pathToDxml}/*.csv")[0]#"/content/drive/MyDrive/security_mft/dfxml_files/*.csv"
print(filepath)
df = pd.read_csv(filepath)
print(f"column names: {df.columns}")
print(df.head(50))

In [ ]:
end_time_total = datetime.now()
print('Duration of loading data: {}'.format(end_time_total - start_time))